# 채권 분석 보고서 크롤링

In [ ]:
# 채권 분석자료 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from IPython.display import clear_output
    
start_page = 1
current_page = start_page
end_page = 126

result_df = pd.DataFrame(columns=['date', 'text'])

while True:
    URL = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('table.type_1 td.file')

    for i, result in enumerate(results):
        pdf_date = sp.select('table.type_1 td.date')[i*2].text
        pdf_temp = requests.get(result.select_one('a').attrs['href'])

        with open('bond.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bond.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df]) 
        
    clear_output(wait=True)
    print('크롤링 {}% 진행 중...'.format(int((current_page/end_page)*100)))
        
    if current_page == end_page:
        clear_output(wait=True)
        print('데이터 크롤링 완료 ! bond_datas.csv에서 확인 가능')
        result_df.to_csv('bond_datas.csv')
        break
    current_page += 1

In [ ]:
# result_df['text'][0]

# 데이터 전처리

In [1]:
import pandas as pd
bond_df = pd.read_csv('bond_datas.csv')
bond_df = bond_df[['date', 'text']]

# 전처리한 것
------------------------
### Header  
Microsoft[^\[\!n\]]* # Header 1  
.*키움증권 리서치센터 투자전략팀 # Header 2 (추가확인필요)  

------------------------
### Footer  
본사·서울시.* # Footer 1  
성명 직급.* # Footer 2  

------------------------
### Notice  
Compliance Notice # Notice  
'(당사는)? ?(동|본) ?(조사|자료).*(없습니다|않습니다|됩니다|바랍니다)\.? # Notice  

------------------------
### etc  
\<[^\<\>]*\> # <>사이의 내용  
그림 [0-9]{1,2}\. # 그림  
표 [0-9]{1,2}\. # 표  
월간채권투자  [0-9]{1,2}월. # page 나누는 기준  
자료: [가-힣a-zA-Z, ]* ? # 자료 출처  
[a-zA-Z0-9\.]*\@[a-zA-Z0-9\.]* # 이메일  

In [3]:
from datetime import date
import datetime
bond_df['date'] = list(map(lambda i : datetime.datetime.strptime(i, "%y.%m.%d").date(), bond_df['date']))

bond_df

,date,text
0,2020-07-15,Microsoft Word - 2020071416393691.docx\n\n\n \...
1,2020-07-15,2013년 0월 0일\n\n\n \n\n \n\n \n\n \n\n \n\n \n\...
2,2020-07-14,(Microsoft Word - 20200714_Market Inside_\277\...
3,2020-07-13,Compliance Notice \n\n \n\n \n\n당사는 동 자료를 기관투...
4,2020-07-10,Microsoft Word - 2020070919412559.docx\n\n\n \...
...,...,...
3747,2008-04-21,Microsoft Word - 920_0421_Fixed Income Weekly....
3748,2008-04-14,Microsoft Word - Fixedincome0414.doc\n\n\n \n\...
3749,2008-04-07,Microsoft Word - 0407.doc\n\n\nFixed Income We...
3750,2008-04-07,Microsoft Word - 0407.doc\n\n\nFixed Income We...


In [4]:
import re

def pp(x):
    x = x.replace('\n','[!n]')
    x = re.sub('Microsoft[^\[\!n\]]*', '', x) # Header 1
    x = re.sub('.*키움증권 리서치센터 투자전략팀', '', x) # Header 2 (추가확인필요)
    x = re.sub('본사·서울시.*', '', x) # Footer 1
    x = re.sub('성명 직급.*', '', x) # Footer 2
    x = re.sub('Compliance Notice', '', x) # Notice
    x = re.sub('(당사는)? ?(동|본) ?(조사|자료).*(없습니다|않습니다|됩니다|바랍니다)\.?', '', x) # Notice
    x = re.sub('\<[^\<\>]*\>', '', x) # <>사이의 내용
    x = re.sub('그림 [0-9]{1,2}\.', '', x) # 그림
    x = re.sub('표 [0-9]{1,2}\.', '', x) # 표
    x = re.sub('월간채권투자  [0-9]{1,2}월.', '', x) # page 나누는 기준
    x = re.sub('자료: [가-힣a-zA-Z, ]* ?', '', x) # 자료 출처
    x = re.sub('[a-zA-Z0-9\.]*\@[a-zA-Z0-9\.]*', '', x) # 이메일
    x = x.replace('[!n]', ' ')
    return x

bond_df['doc'] = list(map(pp, bond_df['text']))

In [5]:
bond_df[bond_df['date'] < datetime.date(2018,1,1)].sort_values(by='date')[['date','doc']].to_json('bond_0517.json')
bond_df[bond_df['date'] > datetime.date(2017,12,31)].sort_values(by='date')[['date','doc']].to_json('bond_1820.json')